# 递归算法

递归是一种解决问题的方法,将问题分解为更小的子问题,直到得到一个足够小的问题可以被很简单的解决。通常递归涉及函数调用自身。递归允许我们编写优雅的解决方案,解决可能很难编程的问题。为了了解递归编程，不妨先从二叉树讲起：

##### 【１】二叉搜索树的创建

给出一个数组alist = [21,28,14,32,25,18,11,30,19,15]，请编写一个算法按照下图示的过程那样创建一颗二叉树：

<img src="image/chapter05/BST.gif" width=400>

从上面可以看出，树的根结点为数组的第一个数，随着后面的值不断插入，这棵树也不断成长。成长规则为：

$\bullet$ 任意节点的左子树不空，则左子树上所有结点的值均小于它的根结点的值；<br>
$\bullet$ 任意节点的右子树不空，则右子树上所有结点的值均大于它的根结点的值；<br>
$\bullet$ 任意节点的左、右子树也分别为二叉查找树；<br>
$\bullet$ 没有键值相等的节点。 

先来定义一个子结点

In [1]:
class Node:
    def __init__(self,root,left=None,right=None):
        self.root = root
        self.left = left
        self.right = right
        return 

然后开始创建树：

In [2]:
def CreatTree(tree,value):
    
    if value>tree.root:
        if tree.right is None:
            tree.right = Node(value)
        else:
            CreatTree(tree.right,value)

    if value<tree.root:
        if tree.left is None:
            tree.left = Node(value)
        else:
            CreatTree(tree.left,value)

    return tree # 一定要返回　tree

##### 【２】理解 CreatTree 递归过程

相信大家早就知道递归的实现是靠堆栈实现的，但是关于递归函数的参数和变量之间的关系却很模糊。为此,我写了几个不同的递归函数来分析它们之间有啥不同！并用一个 main 函数来分析：

In [3]:
def main(CreatTree,alist):
    
    try:   # 使用了一个 try 来处理异常。如果递归正确，那么返回　True;　否则返回 False
        for i in range(len(alist)):
            if i == 0:
                tree=Node(alist[i])
                continue
            tree = CreatTree(tree,alist[i])

        print('     ',tree.root)
        print(' ',tree.left.root,'    ',tree.right.root)
        print(tree.left.left.root,' ',
              tree.left.right.root,tree.right.left.root,' ',tree.right.right.root)
        return True
    
    except:
        print(" Recursion Error...")
        return False

先来看看我编写的 CreatTree 是否正确

In [4]:
alist = [21,28,14,32,25,18,11,30,19,15]
main(CreatTree,alist)

      21
  14      28
11   18 25   32


True

从上面可知，该函数运行正确！

In [5]:
def CreatTree_１(tree,value):
    
    if value>tree.root:
        if tree.right is None:
            tree.right = Node(value)
            return tree
        else:
            tree = CreatTree(tree.right,value) # tree = CreateTree(..) 会覆盖掉原来生成的树
            

    if value<tree.root:
        if tree.left is None:
            tree.left = Node(value)
            return tree
        else:
            tree = CreatTree(tree.left,value)

    return tree 

In [6]:
 main(CreatTree_１,alist)

      19
 Recursion Error...


False

## N进制转换

假设你想将一个整数转换为一个二进制和十六进制字符串。例如,将整数10转换为十进制字符串表示为10,或将其字符串表示为二进制1010。虽然有很多算法来解决这个问题,包括在栈部分讨论的算法,但递归的解决方法非常优雅。

##### 算法

In [7]:
def divideByN(decNumber,N):
    convertString = "0123456789ABCDEF"
    if decNumber < N: # 如果输入值比 N 还小
        return convertString[decNumber] # 那么相应的返回进制字符
    
    else:
        # 否则继续整除，然后继续转换
        return divideByN(decNumber//N,N) + convertString[decNumber%N]

for N in range(2,17):
    ans = divideByN(233,N)
    print("{}进制:{}".format(N,ans))

2进制:11101001
3进制:22122
4进制:3221
5进制:1413
6进制:1025
7进制:452
8进制:351
9进制:278
10进制:233
11进制:1A2
12进制:175
13进制:14C
14进制:129
15进制:108
16进制:E9


## 探索迷宫

迷宫问题的根源与希腊的神话有关,传说忒修斯被送入迷宫中以杀死人身牛头怪。当忒修斯完成杀死野兽的任务，他用了一卷
线帮助他找到回去的退路。在我们的问题中,我们将假设我们的乌龟在迷宫中间的某处,必须找到出路.

<img src="image/chapter03/Screenshot from 2018-03-30 15-29-45.png" width=300>

为了使问题容易些,我们假设我们的迷宫被分成“正方形”。迷宫的每个正方形是开放的或被一段墙壁(用1表示)占据。乌龟只能通过迷宫的空心方块(用0表示)。如果乌龟撞到墙上,它必须尝试不同的方向。

(1) 从我们的起始位置,我们将首先尝试向北一格,然后从那里递归地尝试我们的程序;<br>
(2) 如果我们通过尝试向北作为第一步没有成功,我们将向南一格,并递归地重复我们的程序;<br>
(3) 如果向南也不行,那么我们将尝试向西一格,并递归地重复我们的程序;<br>
(4) 如果北,南和西都没有成功,则应用程序从当前位置递归向东;<br>
(5) 如果这些方向都没有成功,那么没有办法离开迷宫,我们失败。<br>

##### 算法

In [8]:
from copy import deepcopy
from queue import Queue

maze = [
        [1,0,1,1,1,1,1,1,1,1,1,1,1,1],
        [1,0,0,0,1,1,0,0,0,0,0,0,0,1],
        [1,0,1,0,0,0,0,1,0,1,0,1,0,1],
        [1,0,1,0,1,1,1,1,0,1,0,1,0,1],
        [1,0,1,0,0,0,0,0,0,1,1,1,0,1],
        [1,0,1,1,1,1,1,1,0,1,0,0,0,1],
        [1,0,1,0,0,0,0,0,0,0,1,1,0,1],
        [1,0,0,0,1,1,1,0,1,0,1,1,0,1],
        [1,0,1,0,1,0,1,0,1,0,1,0,0,1],
        [1,0,1,0,1,0,1,0,1,0,0,1,0,1],
        [1,0,0,0,0,0,1,0,0,1,0,0,0,1],
        [1,0,1,1,1,0,1,1,0,1,1,0,1,1],
        [1,0,0,0,1,0,0,0,1,0,1,0,1,1],
        [1,1,1,0,0,1,1,0,1,0,0,0,0,1],
        [1,1,1,1,1,1,1,1,1,1,0,1,1,1]]


paths = ([1,0],[-1,0],[0,1],[0,-1]) # 表示东西南北四个方向
answer = Queue() # 用一个队列来存储能够成功走出来的路径
answer.put((0,1))# 起始路径入队列

def find_path(maze,pos):
    
    maze = deepcopy(maze)
    global path,answer
    # 标记该乌龟现在的位置已走过，用 2 表示
    maze[pos[0]][pos[1]] = 2
    # 如果该位置为出口，那么打印足迹，返回 True
    if pos[0] == 14  and pos[1] == 10: return True
    
    # 不是出口，则遍历东南西北四个方向
    for path in paths:
        i,j = pos[0]+path[0],pos[1]+path[1]
        # 寻找可以行走的路
        if maze[i][j] !=0:
            continue
            
        # 现在判断该方向能不能通往出口
        if find_path(maze,[i,j]):
            # 如果能的话，标记该位置走过
             maze[i][j] = 2
             answer.put((i,j)) # 位置入队列
             return True
    
    return False

find_path(maze,[0,1])
while not answer.empty():
    path = answer.get()
    i,j = path
    maze[i][j] = 2

maze

[[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 2, 0, 0, 1, 1, 0, 0, 2, 2, 2, 2, 2, 1],
 [1, 2, 1, 0, 0, 0, 0, 1, 2, 1, 0, 1, 2, 1],
 [1, 2, 1, 0, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1],
 [1, 2, 1, 0, 0, 0, 0, 0, 2, 1, 1, 1, 2, 1],
 [1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 0, 2, 1],
 [1, 2, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 2, 1],
 [1, 2, 0, 2, 1, 1, 1, 0, 1, 0, 1, 1, 2, 1],
 [1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 2, 1],
 [1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 0, 1, 2, 1],
 [1, 2, 2, 2, 0, 0, 1, 0, 0, 1, 0, 2, 2, 1],
 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1],
 [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 2, 1, 1],
 [1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]]

## 八皇后问题

八皇后问题是一个以国际象棋为背景的问题：如何能够在 8×8 的国际象棋棋盘上放置八个皇后，使得任何一个皇后都无法直接吃掉其他的皇后？为了达到此目的，任何两个皇后都不能处于同一条横行、纵行或斜线上。八皇后问题可以推广为更一般的 n 皇后摆放问题：这时棋盘的大小变为 n×n，而皇后个数也变成 n。当且仅当 n = 1 或 n ≥ 4 时问题有解。


<img src="image/chapter03/queue.jpeg" width = 300>

In [9]:
from copy import deepcopy

def Put(chess,pos):
    """
    在棋盘 pos 位置上放了一个皇后，棋盘发生了改变：
    该皇后同一横行、纵行或斜线上的位置都标记为 1 ，
    表示不可以摆放其他皇后了。
    """
    [m,n] = pos
    for i in range(N):
        for j in range(N):
            if abs(i-m) in (0,abs(j-n)) or abs(j-n)==0:
                chess[i][j] = 1
    return chess

def mark(chess,pos,flag):
    """
    棋盘 pos 位置上放了一个皇后(标记为 8 )，并且用
    flag 来记录已经放置在棋盘上的皇后数目。
    """
    chess = Put(chess,pos)
    flag += 1
    [m,n] = pos
    chess[m][n] = 8
    return flag

def find_path(chess,pos,flag):
    chess = deepcopy(chess)  # 进行传参并复制，以免改变原来的棋盘
    flag = mark(chess,pos,flag) # 放置皇后，并标记和记录
    if flag == N:
        for n in range(len(chess)):
            print(chess[n])
        print("Success!")
        return True
    
    # 遍历棋盘上可以放置皇后的位置
    for i in range(N):
        for j in range(N):
            if chess[i][j] == 0:
                # 如果可以成功，那么返回True， 否则继续遍历
                if find_path(chess,[i,j],flag):
                    pos = [i,j]
                    flag = mark(chess,pos,flag)
                    return True
    return False

N = 8 # 
chess = [[0]*N for i in range(N)]
find_path(chess,[0,0],0)

[8, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 8, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 8]
[1, 1, 1, 1, 1, 8, 1, 1]
[1, 1, 8, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 8, 1]
[1, 8, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 8, 1, 1, 1, 1]
Success!


True